In [1]:
import json
import numpy as np
from timeit import default_timer

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

C:\Program Files\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(Flatten(input_shape=(1, 16)))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(3, activation='linear'))

In [3]:
from keras import losses

model.compile(Adam(lr=1e-3), metrics=['mae', 'accuracy'], loss=losses.mean_squared_error)

In [4]:
import json
import math
import os
from tqdm import tqdm_notebook

def norm_angle(theta):
    ''' Normalize an angle in radians to [0, 2*pi) '''
    angle = theta % (2*math.pi)
    if angle < 0:
        angle = 2*math.pi - angle
    return angle

prev_pos = (-1, -1)
prev_angle = -1


def get_observation(msg):
    global prev_pos, prev_angle
    observation = []
    for car in ['my_car', 'enemy_car']:
        data = msg[car]
        # print('angle ({}): {}'.format(car, data[1], norm_angle(data[1])))
        if data[2] == -1:
            for i in [0, 3, 4]:
                data[i] = (1200 - data[i][0], data[i][1])
        observation.extend([data[2], norm_angle(data[1]), data[3][0], data[3][1], data[4][0], data[4][1]])
    observation.append(10)#min(observation[3], observation[5]) - msg['params'].get('deadline_position', 0))
    if prev_pos[0] == -1:
        observation.extend([0.0, 0.0])
    else:
        observation.extend([(observation[2] + observation[4]) / 2 - prev_pos[0],
                            (observation[3] + observation[5]) / 2 - prev_pos[1]])
    prev_pos = ((observation[2] + observation[4]) / 2, (observation[3] + observation[5]) / 2)
    if prev_angle == -1:
        observation.append(0.0)
    else:
        observation.append(observation[1] - prev_angle)
    prev_angle = observation[1]

    return observation


def process(directory):
    global prev_pos, prev_angle
    # [0,"stop",[[300.0,300.0],0.0,1,[329.0,295.0,0.0],[422.0,295.0,0.0]]]
    states = []
    actions = []
    terminal = []
    reward = []
    for fname in tqdm_notebook(os.listdir(directory)):
        with open(directory + '/' + fname) as f:
            prev_pos = (-1, -1)
            prev_angle = -1
            raw_my = f.readline()
            raw_enemy = f.readline()
            try:
                raw_my = json.loads(raw_my)
            except:
                print('raw_my', raw_my)
                continue
            try:
                raw_enemy = json.loads(raw_enemy)
            except:
                print('raw_enemy', raw_enemy)
                continue
            if not raw_my['win']:
                print('Skipping {}...'.format(fname))
                continue
                #raw_my, raw_enemy = raw_enemy, raw_my
            obs_my, obs_enemy = raw_my['dump'], raw_enemy['dump']
            for step in range(len(obs_my)):
                actions.append(obs_my[step][1])
                states.append(get_observation({'my_car': obs_my[step][2], 'enemy_car': obs_enemy[step][2]}))
                if step < len(obs_my) - 1:
                    terminal.append(False)
                    reward.append(1)
                else:
                    terminal.append(True)
                    reward.append(10000)
    return states, actions, terminal, reward

In [44]:
states, actions, terminal, reward = process('dumps/PillCarcassMap_Buggy')

Skipping dump_11832907.dat...
Skipping dump_12293293.dat...
Skipping dump_13006602.dat...
Skipping dump_14486439.dat...
Skipping dump_15990758.dat...
Skipping dump_16203432.dat...
Skipping dump_17161855.dat...
Skipping dump_17986139.dat...
Skipping dump_18271944.dat...
Skipping dump_18410106.dat...
Skipping dump_1883096.dat...
Skipping dump_19217954.dat...
Skipping dump_19282197.dat...
Skipping dump_2041416.dat...
Skipping dump_24998655.dat...
Skipping dump_25956862.dat...
Skipping dump_27936358.dat...
Skipping dump_28430348.dat...
Skipping dump_30585970.dat...
Skipping dump_31683463.dat...
Skipping dump_32319065.dat...
Skipping dump_34909062.dat...
Skipping dump_35306096.dat...
Skipping dump_35939991.dat...
Skipping dump_36463644.dat...
Skipping dump_39022448.dat...
Skipping dump_40783271.dat...
Skipping dump_43263294.dat...
Skipping dump_43924344.dat...
Skipping dump_45142051.dat...
Skipping dump_46719636.dat...
Skipping dump_47059906.dat...
Skipping dump_48858393.dat...
Skipping dum

In [45]:
len(states)

22595

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                544       
_________________________________________________________________
activation_7 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_8 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
activation_9 (Activation)    (None, 32)                0         
__________

In [46]:
from keras.utils import to_categorical

x_train = np.array(states)

atoi = {'left': 0, 'right': 1, 'stop': 2}
y_train = to_categorical([atoi[a] for a in actions])

In [47]:
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))

In [9]:
x_train

array([[[ 1.00000000e+00,  0.00000000e+00,  3.38000000e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 1.00000000e+00,  0.00000000e+00,  3.38000000e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 1.00000000e+00,  6.28066890e+00,  3.37923566e+02, ...,
         -7.63717844e-02, -1.88966282e-01,  6.28066890e+00]],

       ...,

       [[ 1.00000000e+00,  7.69124977e-01,  5.67737875e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 1.00000000e+00,  7.69124977e-01,  5.67737875e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 1.00000000e+00,  7.69124977e-01,  5.67737875e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]])

In [48]:
model.fit(x=x_train, y=y_train, batch_size=4, epochs=20)

Epoch 1/20
22595/22595 [==============================] - 10s 445us/step - loss: 0.1121 - mean_absolute_error: 0.2293 - acc: 0.7660
Epoch 2/20
22595/22595 [==============================] - 10s 450us/step - loss: 0.0993 - mean_absolute_error: 0.2054 - acc: 0.7845
Epoch 3/20
22595/22595 [==============================] - 10s 438us/step - loss: 0.0970 - mean_absolute_error: 0.2013 - acc: 0.7926
Epoch 4/20
22595/22595 [==============================] - 9s 408us/step - loss: 0.0930 - mean_absolute_error: 0.1943 - acc: 0.8031
Epoch 5/20
22595/22595 [==============================] - 10s 422us/step - loss: 0.0903 - mean_absolute_error: 0.1902 - acc: 0.8142
Epoch 6/20
22595/22595 [==============================] - 10s 460us/step - loss: 0.0888 - mean_absolute_error: 0.1867 - acc: 0.8161
Epoch 7/20
22595/22595 [==============================] - 9s 411us/step - loss: 0.0879 - mean_absolute_error: 0.1850 - acc: 0.8185
Epoch 8/20
22595/22595 [==============================] - 9s 415us/step - loss

In [70]:
model.predict(x_train, batch_size=1)

array([[0.06426321, 0.22319226, 0.7107716 ],
       [0.06426321, 0.22319226, 0.7107716 ],
       [0.06426321, 0.22319226, 0.7107716 ],
       ...,
       [0.2116197 , 0.18295498, 0.6005717 ],
       [0.10605744, 0.21177989, 0.67951596],
       [0.06426321, 0.22319226, 0.7107716 ]], dtype=float32)

In [65]:
x_train[0][0]

array([  1.,   0., 329., 295., 422., 295.,  -1.,   0., 329., 295., 422.,
       295.,  10.,   0.,   0.,   0.])

In [49]:
model.save_weights('duel_dqn_101.h5f')